In [2]:
import os
import numpy as np
import datetime
import pandas as pd
import re
import json
from random import sample

In [7]:
def search(start_year,form_name,last_year):
    for i in range (0,last_year):
        Targets_table=("C:/Users/WenlongZhang/Desktop/NLP/datasets/data/"+str(start_year)+form_name+".csv")
        Table=(str(start_year)+form_name+".csv")
        start_year=start_year-1        
    return Targets_table    

In [8]:
def vax_name(field):
    value = []
    for x in field:
        x = re.sub(r'\s{1}\(.*\)', '', x)
        x = x.split(' + ')
        value += x[:]
    value = list(set(value))
    return value


In [9]:
def build_lookup(data, vax, symp):
    datasets = []
    for f in [data, vax, symp]:
        df = pd.read_csv(f, encoding = 'Windows-1252')
        datasets.append(df)
        
    text_fields = ['LAB_DATA','OTHER_MEDS', 'CUR_ILL', 'HISTORY', 'PRIOR_VAX','ALLERGIES']
    symp_list = ['SYMPTOM1', 'SYMPTOM2','SYMPTOM3', 'SYMPTOM4', 'SYMPTOM5']
    lookup_dict = {}
    SYMPTOM = []
    
    for dataset in datasets:
        cols = list(dataset)
        cols.remove('VAERS_ID')
        for x in cols:
            value = []
            if (x not in text_fields) and (x != 'SYMPTOM_TEXT'):
                value = dataset[x].dropna().unique().tolist()
                lookup_dict[x] = value
            if x in symp_list:
                temp = dataset[x].dropna().unique().tolist()
                SYMPTOM += temp 
        SYMPTOM = list(set(SYMPTOM))
    lookup_dict['SYMPTOM'] = SYMPTOM
    lookup_dict['SYMPTOM_TXT'] = SYMPTOM
    lookup_dict['PRIOR_VAX'] = vax_name(lookup_dict['VAX_NAME'])
    return lookup_dict  


lookup2021 = build_lookup(data=search(2021,"VAERSDATA",1), vax=search(2021,"VAERSVAX",1), symp=search(2021,"VAERSSYMPTOMS",1))
fout = open("lookup2021.json", "w")
json.dump(lookup2021, fout)
fout.close()

C:\Users\WenlongZhang\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3338: DtypeWarning: Columns (7,12,15,23) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [10]:
df_lt = pd.read_excel("Text2ESQ_lt.xlsx")
def randsample(lookup_list):
    # random sample the lookup table value based on the total number of value in one field 
    if len(lookup_list) <= 100:
        value_list = lookup_list
    elif len(lookup_list) > 100 and len(lookup_list) <= 300 :
        value_list = sample(lookup_list, 100)
    elif len(lookup_list) > 300:
        value_list = sample(lookup_list, 200)
    return value_list


def replace_value(q, lookup_table):
    pattern = re.compile(r'\[(.*?)\]', re.S)
    questions = []
    
    # find the key value
    q_key = re.findall(pattern, q)
    
    # based on the question template, there are three senarios
    # only one field involved
    if len(q_key) == 1:
        for v in randsample(lookup_table[q_key[0]]):
            temp = re.sub(r"\[(.*?)\]", lambda x: str(v),q)
            questions.append(temp)
    
    # two fields involved
    elif len(q_key) == 2:
        for v in randsample(lookup_table[q_key[0]]):
            temp = re.sub(r"\[(.*?)\]", lambda x: str(v), q, count=1)
            temp = re.sub(r"\[(.*?)\]", lambda x: str(sample(lookup_table[q_key[1]],1)),temp)
            questions.append(temp)
    
    # no need to replace the value 
    elif len(q_key) == 0:
        questions.append(q)
    
    return questions


def create_id(question, query, index):
    
    # generate id with a format of "Question template id" + "Query template id" + "Index"
    # example: "01100001" 
    # the first two digits "01" indicates the question template id 1
    # the third digit "1" indicates the query template id 1
    # the last five digits "00001" indicates the index of the question 1
    
    question = str(question)
    query = str(query)
    index = str(index)
    
    if len(question) == 1:
        question = '0' + question
    
    diff = 5 - len(index)
    if diff < 5:
        index = '0'*diff + index
    
    output = question + query + index
    
    return output


def populate_questions(dataset, lookup_table):
    # Split the question tempalte, explode data, and filter out easy questions
    df = dataset.copy()
    df.Questions = df.Questions.apply(lambda x: x.split("##"))
    df = df.explode('Questions').reset_index().drop(columns='index')
    df = df[df['Difficulty'] == 'Easy']
    
    # Replace square braskets with value, explode data, and create id
    df.Questions = df.Questions.apply(lambda x: replace_value(x, lookup_table))
    df = df.explode('Questions').reset_index().drop(columns='index')
    df['index'] = df.index.values
    df['Query Template ID'] = df['Query Template ID'].apply(lambda x: int(x))
    df['id'] = list(map(lambda x, y, z: create_id(x, y, z), \
                        df['Question Template ID'], \
                        df['Query Template ID'], \
                        df['index']))
    
    return df

questions = populate_questions(df_lt, lookup2021)
print(questions)


      Question Template ID      Data File  \
0                        1      VAERSDATA   
1                        1      VAERSDATA   
2                        1      VAERSDATA   
3                        1      VAERSDATA   
4                        1      VAERSDATA   
...                    ...            ...   
8363                    49  VAERSSYMPTOMS   
8364                    49  VAERSSYMPTOMS   
8365                    49  VAERSSYMPTOMS   
8366                    49  VAERSSYMPTOMS   
8367                    49  VAERSSYMPTOMS   

                                             Field  \
0                                         RECVDATE   
1                                         RECVDATE   
2                                         RECVDATE   
3                                         RECVDATE   
4                                         RECVDATE   
...                                            ...   
8363  SYMPTOM1，SYMPTOM2，SYMPTOM3,SYMPTOM4，SYMPTOM5   
8364  SYMPTOM1，SYMPTOM2，SYM

In [16]:
df = pd.read_excel('Template-Text2ESQ.xlsx')
pd.DataFrame(df)
left=df[['Field','Question Template ID','Data File']]
df1 = questions
right=df1[['Field','Questions']]
df_x = pd.merge(left,right)
js_x = pd.merge(left,right).to_json(orient = 'index')
print(js_x)

{"0":{"Field":"RECVDATE","Question Template ID":1,"Data File":"VAERSDATA","Questions":"Give me all the patients whose information are received on 11\/04\/2021. "},"1":{"Field":"RECVDATE","Question Template ID":1,"Data File":"VAERSDATA","Questions":"Give me all the patients whose information are received on 06\/25\/2021. "},"2":{"Field":"RECVDATE","Question Template ID":1,"Data File":"VAERSDATA","Questions":"Give me all the patients whose information are received on 05\/08\/2021. "},"3":{"Field":"RECVDATE","Question Template ID":1,"Data File":"VAERSDATA","Questions":"Give me all the patients whose information are received on 05\/17\/2021. "},"4":{"Field":"RECVDATE","Question Template ID":1,"Data File":"VAERSDATA","Questions":"Give me all the patients whose information are received on 02\/28\/2021. "},"5":{"Field":"RECVDATE","Question Template ID":1,"Data File":"VAERSDATA","Questions":"Give me all the patients whose information are received on 10\/12\/2021. "},"6":{"Field":"RECVDATE","Qu

In [15]:
df = pd.read_excel('Template-Text2ESQ.xlsx')
pd.DataFrame(df)
left=df[['Field','Question Template ID','Data File']]
df1 = pd.read_json('Questions.json')

right=df1[['Field','Question']]
df_p = pd.merge(left,right)
js_p = pd.merge(left,right).to_json(orient = 'index')

str

In [19]:
frames = [df_x, df_p]
result = pd.concat(frames,ignore_index=True)

print(result)

           Field  Question Template ID  Data File  \
0       RECVDATE                     1  VAERSDATA   
1       RECVDATE                     1  VAERSDATA   
2       RECVDATE                     1  VAERSDATA   
3       RECVDATE                     1  VAERSDATA   
4       RECVDATE                     1  VAERSDATA   
...          ...                   ...        ...   
11063  PRIOR_VAX                    33  VAERSDATA   
11064  PRIOR_VAX                    33  VAERSDATA   
11065  PRIOR_VAX                    33  VAERSDATA   
11066  PRIOR_VAX                    33  VAERSDATA   
11067  PRIOR_VAX                    33  VAERSDATA   

                                               Questions  \
0      Give me all the patients whose information are...   
1      Give me all the patients whose information are...   
2      Give me all the patients whose information are...   
3      Give me all the patients whose information are...   
4      Give me all the patients whose information are...   
...